# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [1]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

[nltk_data] Downloading package punkt to /home/assil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [3]:
# FOR TESTING : only select first 20000 samples
X, y = X[:20000], y[:20000]

In [4]:
tokenized_documents = tokenizer(X)

In [5]:
X, vect = vectorizer(tokenized_documents)

/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0003' '000kwh' '002' '008' '01' '0100' '0174']
Top 10 least frequent words in the dataset
['zuke' 'zukes' 'zupas' 'zuppa' 'zwieback' 'º' '¼' '½' 'çaykur' 'ît']


In [7]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(16000, 26658) (4000, 26658) (16000,) (4000,)
  (0, 20554)	0.3845362652831886
  (0, 6390)	0.3436631403149654
  (0, 14429)	0.3724722072082089
  (0, 4235)	0.20728370948487648
  (0, 17569)	0.2629960113653433
  (0, 23859)	0.15524178896215457
  (0, 11671)	0.2679384409265626
  (0, 8352)	0.23213222149909707
  (0, 21911)	0.29415770046654777
  (0, 16681)	0.19747406219111086
  (0, 1912)	0.14002887633461686
  (0, 26519)	0.14117053473926772
  (0, 26200)	0.13006169326147435
  (0, 23803)	0.12657510519567464
  (0, 841)	0.08667788302140157
  (0, 10901)	0.15433885114321025
  (0, 23821)	0.16630262302915325
  (0, 1653)	0.164344730408501
  (0, 23811)	0.15903521582936178
  (0, 11595)	0.13181371070406692
  (0, 12252)	0.08195440863309632


# Feed Forward Neural Network
### Model starts here

In [9]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

2024-04-18 16:19:18.013408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 16:19:19.401805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Assuming X_train and X_test are loaded
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Number of samples and features
num_samples = X_train_dense.shape[0]
total_features = X_train_dense.shape[1]

# Choosing timesteps (make sure this is a divisor of total_features)
timesteps = 1  # Example: Adjust this based on your actual data structure
if total_features % timesteps != 0:
    raise ValueError("Timesteps must divide the total number of features exactly.")

features_per_timestep = total_features // timesteps

# Reshape data
X_train_lstm = X_train_dense.reshape(num_samples, timesteps, features_per_timestep)
X_test_lstm = X_test_dense.reshape(-1, timesteps, features_per_timestep)

# Convert labels to categorical format
num_classes = np.max(y_train) + 1
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# Define the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(timesteps, features_per_timestep), activation='relu', return_sequences=True))
model.add(LSTM(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_lstm, y_train_cat, epochs=10, batch_size=32)

2024-04-18 16:21:10.157522: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 16:21:10.538950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 16:21:10.539172: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/10


I0000 00:00:1713450076.186918    7747 service.cc:145] XLA service 0x7338580038d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713450076.186970    7747 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-04-18 16:21:16.259174: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-18 16:21:16.582712: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 19/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5299 - loss: 1.7579

I0000 00:00:1713450077.583147    7747 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.6291 - loss: 1.1359
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7345 - loss: 0.6657
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8394 - loss: 0.4148
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9308 - loss: 0.2101
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9726 - loss: 0.0829
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9952 - loss: 0.0205
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9981 - loss: 0.0104
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9993 - loss: 0.0036
Epoch 9/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9997 - loss: 0.0034
Epoch 10/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9995 - loss: 9.3536e-04


In [14]:
# Evaluate the model
predictions_prob = model.predict(X_test_lstm)
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
from sklearn import metrics
print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[[ 171   67   47   11   57]
 [  41   85   68   20   32]
 [  22   49  142   54   75]
 [   9   20   95  182  261]
 [  49   26  147  257 2013]]
              precision    recall  f1-score   support

           1       0.59      0.48      0.53       353
           2       0.34      0.35      0.34       246
           3       0.28      0.42      0.34       342
           4       0.35      0.32      0.33       567
           5       0.83      0.81      0.82      2492

    accuracy                           0.65      4000
   macro avg       0.48      0.47      0.47      4000
weighted avg       0.66      0.65      0.65      4000



In [ ]:
# Save the model with current date and time in model folder
import datetime

# Create a folder named _models in the current directory
import os
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [17]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(-1, timesteps, features_per_timestep)
    result = model.predict(test_vec_lstm)
    predicted_score = result.argmax() + 1
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
Test sentence: This is a good product
Predicted score: 5

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Test sentence: This is a bad product
Predicted score: 2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Test sentence: This is a product
Predicted score: 4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Test sentence: This is a very good product
Predicted score: 6

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Test sentence: This is a very bad product
Predicted score: 2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Test sentence: That was bad
Predicted score: 2

